In [23]:
:opt no-lint

In [1]:
import Data.Monoid
import Data.Foldable

#  20 Foldable
## 20.1 Foldable
content:
- The `Foldable` class which is kind of generalized list
- The monoidal nature of folding.
- Standard operations derived from folding (like mapping and filtering)

## 20.2 The Foldable class
`Foldable` is a “class of data structures that can be folded to a summary value.”
A minimal instance define either `foldMap` or `foldr`.
## 20.3 Revenge of the monoids
Folding necessarily implies a binary associative operation that has an identity value.
- `fold` uses `Monoid` to combine elements inside a `Foldable` structure:

In [4]:
:t fold
fold ["Hello", ",", " ", "World", "!"]

fold :: forall (t :: * -> *) m. (Foldable t, Monoid m) => t m -> m

"Hello, World!"

- `foldMap` maps elements of the structure to a `Monoid` and then combines the results using that instance of `Monoid`:

In [5]:
:t foldMap
foldMap ((*2) . Sum) [1..5]
foldMap ((*3) . Sum) (Just 2)

foldMap :: forall (t :: * -> *) m a. (Foldable t, Monoid m) => (a -> m) -> t a -> m

Sum {getSum = 30}

Sum {getSum = 6}

## 20.4 Demonstrating Foldable instances
### Identity
The `Identity` catamorphism consists in practice in consuming the value inside the structure.
### Maybe
The `Maybe` catamorphism is like `Identify` when applyed to a `Just`. When applyed to a `Nothing`, it essentially ignore it.
## 20.5 Some basic derived operations
The `Foldable` type class includes other functions that generalize list operations: `toList`, `null`, `length`, `elem`, `maximum`, `minimum`, `sum`, `product`.
### Exercises: Library functions
Implement the functions in terms of `foldMap` or `foldr` from `Foldable`, and apply them out with multiple types that have `Foldable` instances.
1. This and the next one are nicer with foldMap, but the foldr function is fine, too:

In [3]:
sum :: (Foldable t, Num a) => t a -> a
sum = getSum . foldMap Sum

sum [1..5]
sum Nothing
sum (7, 5)
sum (Just 1)
fmap sum [(7, 5), (3, 4)]
fmap sum (Just [1, 2, 3, 4, 5])
sum (Left "error")
sum (Right 42)

15

0

5

1

[5,4]

Just 15

0

42

2. 

In [7]:
product :: (Foldable t, Num a) => t a -> a
product = foldr (*) 1

product [1..5]
fmap product (Just [])
fmap product (Right [1, 2, 3])

120

Just 1

Right 6

3. 

In [4]:
elem :: (Foldable t, Eq a) => a -> t a -> Bool
elem x = getAny . foldMap (Any . (==x))

3 `elem` [1..10]
2 `elem` Just 3
True `elem` Left False
True `elem` Left True
True `elem` Right False
True `elem` Right True
42 `elem` [1..]

True

False

False

False

False

True

True

4. 

In [4]:
import Data.Semigroup

minimum :: (Foldable t, Ord a) => t a -> Maybe a
minimum =  fmap getMin . foldMap (Just . Min)

minimum [1..5]
minimum "julie"
minimum (Just 4)
minimum Nothing
minimum (Right 42)

Just 1

Just 'e'

Just 4

Nothing

Just 42

5. 

In [10]:
maximum :: (Foldable t, Ord a) => t a -> Maybe a
maximum = foldr reducer Nothing
  where
    reducer x Nothing = Just x
    reducer x (Just y) = Just $ max x y

maximum [1..5]
maximum (Just 10)
maximum (Left '👅')
maximum []

Just 5

Just 10

Nothing

Nothing

6. 

In [24]:
null :: (Foldable t) => t a -> Bool
null = foldr ((const . const) False) True 

null (Left 3)
null []
null Nothing
null (1, 2)
null [1..]

True

True

True

False

False

7. 

In [12]:
length :: (Foldable t) => t a -> Int
length = foldr (const (+ 1)) 0

length (1, 2)
length [(1, 2), (3, 4), (5, 6)]
length $ Just [1, 2, 3]
fmap length Just [1, 2, 3]

1

3

1

1

8. Some say this is all `Foldable` amounts to:

In [13]:
toList :: (Foldable t) => t a -> [a]
toList = foldMap pure

toList (Just 1)
toList (1, 2)
toList (Left "error")

[1]

[2]

[]

9. Define `fold` in terms of `foldMap`:

In [14]:
fold :: (Foldable t, Monoid m) => t m -> m
fold = foldMap id

fold ["Hello", ",", " ", "World", "!"]
fold (Just "👽")

"Hello, World!"

"\128125"

10. Define `foldMap` in terms of `foldr`:

In [17]:
foldMap :: (Foldable t, Monoid m) => (a -> m) -> t a -> m
foldMap f = foldr ((<>) . f) mempty

foldMap ((*4) . Sum) [1..5]
appEndo (foldMap Endo [(+1), (*2), negate]) 5

Sum {getSum = 60}

-9

## 20.6 Chapter exercises
Write `Foldable` instances for the following datatypes:
1. 

In [16]:
data Constant a b = Constant b

instance Foldable (Constant a) where
  foldr f z (Constant x) = f x z

foldr (+) 3 (Constant 10)
foldMap Sum (Constant 42)

13

Sum {getSum = 42}

2. 

In [17]:
data Two a b = Two a b

instance Foldable (Two a) where
  foldMap f (Two _ x') = f x'

foldr (+) 3 (Two '👾' 10)
foldMap Sum (Two '👍' 42)

13

Sum {getSum = 42}

3. 

In [18]:
data Three a b c = Three a b c

instance Foldable (Three a b) where
  foldr f z (Three _ x' x'') = f x'' z

foldr (+) 3 (Three '👾' (+1) 10)
foldMap Sum (Three '👍' (*2) 42)

13

Sum {getSum = 42}

4. 

In [19]:
data Three' a b = Three' a b b

instance Foldable (Three' a) where
  foldMap f (Three' _ x' x'') = f x' <> f x''

foldr (+) 3 (Three' '👾' 1 10)
foldMap Sum (Three' '👍' 2 42)

14

Sum {getSum = 44}

5. 

In [20]:
data Four' a b = Four' a b b b

instance Foldable (Four' a) where
  foldMap f (Four' _ x' x'' x''') = f x' <> f x'' <> f x'''

foldr (++) " Stella!" (Four' "👾" "1" "2" "3")
foldMap Sum (Four' '🌟' 1 2 3)

"123 Stella!"

Sum {getSum = 6}

Write a filter function for `Foldable` types using the `foldMap` function:

In [22]:
filterF :: (Applicative f, Foldable t, Monoid (f a)) => (a -> Bool) -> t a -> f a
filterF f = foldMap (\x -> if f x then pure x else mempty)

length $ filterF (>3) [1..10]
length $ filterF (>3) Nothing
length $ filterF (>3) (Just 1)
length $ filterF (>3) (Just 10)

7

0

0

1

`Foldable` only allow to "read", in order to build a new structure we need `Applicative` and `Monoid`.